In [1]:
import sys, os, fnmatch, datetime, subprocess, imp
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
fpath = os.getcwd()

In [8]:
lib = pd.read_csv(fpath+'/061318_exonskipping_library.csv', index_col=0, usecols = ('Identifier number', 'Designed 61-bp target site (37i-24e, AG)')).T.to_dict()

In [10]:
gene_lib = pd.read_csv(fpath+'/061318_exonskipping_library.csv', index_col = 0, usecols = ('Identifier number', 'gene_symbol')).T.to_dict()

In [ ]:
b3 = pd.DataFrame(columns = ('umi', 'id'))
b6 = pd.DataFrame(columns = ('umi', 'id', 'slice_type', 'num'))
b7 = pd.DataFrame(columns = ('umi', 'id'))

In [ ]:
for root, subdirs, files in os.walk('/home/clara/Desktop/data/post_cas9_mesc'):
    fpath = '/home/clara/Desktop/data/post_cas9_mesc/'
    for subdir in subdirs:
        if subdir is not "0722_cas9_treated.ipynb" and 'b6' not in subdir and 'checkpoints' not in subdir and subdir is not '0725_cas9_untreated.ipynb' and 'mesc' not in subdir:
            print(subdir)
            if int(subdir) in [0,1,2,4,8,9,13] :
                subdir = subdir+'/'
                sub3 = pd.read_csv(fpath+subdir+'b3', names = ('umi', 'id'), dtype={'id':'Int64'})
                sub6 = pd.read_csv(fpath+subdir+'b6', names = ('umi', 'id', 'slice_type', 'num'), dtype={'id':'Int64'})
                sub7 = pd.read_csv(fpath+subdir+'b7', names = ('umi', 'id'), dtype={'id':'Int64'})
                b3 = pd.concat([b3, sub3])
                b6 = pd.concat([b6, sub6])
                b7 = pd.concat([b7, sub7])
            elif int(subdir)!=5 and int(subdir)!=14:
                subdir = subdir+'/'
                sub3 = pd.read_csv(fpath+subdir+'b3', names = ('umi', 'id'), dtype={'id':'Int64'})
                sub7 = pd.read_csv(fpath+subdir+'b7', names = ('umi', 'id'), dtype={'id':'Int64'})
                b3 = pd.concat([b3, sub3])
                b7 = pd.concat([b7, sub7])

In [25]:
match_dct = {
    'A':'T',
    'C':'G',
    'T':'A',
    'G':'C',
}

In [47]:
b6.to_csv('b6.csv', index=False)
b3.to_csv('b3.csv', index=False)
b7.to_csv('b7.csv', index=False)

In [46]:
b6 = b6.dropna()
b6.slice_type = b6.slice_type.apply(lambda x: x.strip())
b6 = b6[~b6.umi.str.contains("N")]
b6.umi = b6.umi.apply(lambda x: remove_ggc(x))

In [13]:
b6 = pd.read_csv(fpath+'/b6.csv')
b3 = pd.read_csv(fpath+'/b3.csv')
b7 = pd.read_csv(fpath+'/b7.csv')

In [23]:
b7_nodups = b7.dropna()
b7_nodups = b7_nodups[~b7_nodups.umi.str.contains("N")]

## __B7 Analysis Genomic Data__ [gRNA and UMI dictionary]


In [14]:
def reverse_complement(umi):
    if len(umi) == 0:
        return ''
    try:
        this = match_dct[umi[-1:]]
    except:
        print(umi)
        this = ''
    return this + reverse_complement(umi[:-1])

In [26]:
b7_nodups = b7.dropna()
b7_nodups = b7_nodups[~b7_nodups.umi.str.contains("N")]
b7_nodups.umi = b7_nodups['umi'].apply(lambda x: reverse_complement(x))

In [27]:
b7_uniq_count = b7_nodups.groupby('umi')['id'].nunique().reset_index('umi').rename(columns = {'id':'id_count'})
b7_uniq_umi = b7_uniq_count.loc[b7_uniq_count.id_count==1]

In [28]:
b7_match = b7_nodups[b7_nodups.umi.isin(b7_uniq_umi['umi'])]

b7_match contains read information, unique matches from umi to id. umis that match to more than one id were eliminated above

In [29]:
b7_nonunique = b7_uniq_count.loc[b7_uniq_count.id_count > 1]
b7_nonmatch = b7_nodups[b7_nodups.umi.isin(b7_nonunique['umi'])]
b7_percentages = b7_nonmatch.groupby('umi')['id'].value_counts(normalize=True)

In [30]:
b7_percentages = pd.DataFrame(b7_percentages).rename(columns = {'id':'frequency'})

Below: df of umis that predominately map to 1 id, filtered from umis that map to 2 or more ids

In [31]:
b7_rematch = b7_percentages.loc[b7_percentages.frequency > 0.8].reset_index('umi').reset_index('id')

In [ ]:
b7_rematch_dict = pd.Series(b7_rematch.id.values, index = b7_rematch.umi).to_dict()

In [ ]:
b7_nodups['in_rematch'] = b7_nodups.umi.apply(lambda x: True if x in b7_rematch_dict else False)
b7_rematch_read = b7_nodups.loc[b7_nodups.in_rematch == True]

In [ ]:
def dict_match(umi, ID):
    return b7_rematch_dict[umi] == ID

In [ ]:
b7_rematch_read['correct_id'] = b7_rematch_read.apply(lambda x: dict_match(x.umi, x.id),axis=1)

In [ ]:
b7_rematch_read = b7_rematch_read.loc[b7_rematch_read.correct_id == True]

In [ ]:
b7_complete = pd.concat([b7_rematch_read, b7_match])

In [ ]:
b7.id.nunique()

In [34]:
b7_complete = pd.read_csv(fpath+'/b7_complete.csv')

In [35]:
b7_dict = pd.Series(b7_complete.id.values, index=b7_complete.umi).to_dict()

In [ ]:
b7_complete.to_csv('b7_complete.csv', columns=['umi','id'], index = False)

Confirm that matches in b7_match only match to one id - shallow level confirmation, just checking one umi

In [ ]:
b7_match.loc[b7_match.umi == 'AAGGTCTGACTGTCG']

In [ ]:
def genomic_data(df):
    #assumes that we do not want read count variation information
    df_nodup = df.drop_duplicates().dropna()
    df_nodup = df_nodup[~df_nodup.umi.str.contains("N")]
    df_nodup.umi = df_nodup.umi.apply(lambda x: reverse_complement(x))
    df_uniq_count = df_nodup.groupby('umi').count()
    df_uniq_count = df_uniq_count.loc[df_uniq_count.id==1].rename(columns={'id':'count'})
    df_uniq_umi = pd.merge(df_nodup,df_uniq_count, how='inner', on=['umi'])
    print('number of unique ids:' + str(b7_uniq_umi.id.nunique()))
    print('\n'+'are the number of unique umis equal to the number of rows in df? '+str(b7_uniq_umi.umi.nunique() == b7_uniq_umi.shape[0]))
    return df_uniq_umi

#### Q: For each duplicated UMI, is it the case that most of the IDs matched to it are of one number, and only a few are of another? (i.e. sequencing or small recombo error)

Yes for some - check TATAGCCTTTTTTTT for an example

#### Q: What the benchmark for recapturing a UMI that was previously mapped to more than one ID?

3 standard deviations? average number of reads per umi in the nonmatch dataset
- ask Ben

## __B3 analysis__ [unspliced data]

In [ ]:
b3_nodups = b3.dropna()
b3_nodups = b3_nodups[~b3_nodups.umi.str.contains("N")]

# removed non unique matches first --> should we not do this?? - this doens't do much --> seems like the duplications are evenly distributed across matches and nonmatches
b3_uniq_count = b3_nodups.groupby('umi')['id'].nunique().reset_index('umi').rename(columns = {'id':'id_count'})
#b3_uniq_count has umi and id_count column - each umi appears once with its matched id count 
b3_uniq_umi = b3_uniq_count.loc[b3_uniq_count.id_count==1]
#b3_uniq_umi contains each unique umi with only one for id_count 
b3_unique = b3_nodups[b3_nodups.umi.isin(b3_uniq_umi['umi'])]
#b3_unique contains read duplicates

In [ ]:
b3_unique.umi.nunique()

Removed nonunique matches --> restored matches that occurred with > 80% probability between on particular UMI - ID pairing

In [ ]:
b3_nonunique = b3_uniq_count.loc[b3_uniq_count.id_count > 1] 
b3_non_uniq = b3_nodups[b3_nodups.umi.isin(b3_nonunique['umi'])] #b3_unique + b3_non_uniq count == b3 count
b3_percentages = b3_non_uniq.groupby('umi')['id'].value_counts(normalize=True)
b3_percentages = pd.DataFrame(b3_percentages).rename(columns = {'id':'frequency'})
b3_rematch = b3_percentages.loc[b3_percentages.frequency > 0.8].reset_index('umi').reset_index('id') #only contains each umi and id pairing once --> doesn't reflect read count 
b3_rematch = b3_rematch.drop('frequency', axis=1)

In [ ]:
b3_rematch_dict = pd.Series(b3_rematch.id.values, index = b3_rematch.umi).to_dict()

b3_non_uniq['in_rematch'] = b3_non_uniq.umi.apply(lambda x: True if x in b3_rematch_dict else False)
b3_rematch_read = b3_non_uniq.loc[b3_non_uniq.in_rematch == True]
b3_rematch_read = b3_rematch_read.drop('in_rematch', axis=1)

b3_join = b3_rematch_read.join(pd.Series(b3_rematch.id.values, index = b3_rematch.umi).rename('dict'), on='umi')
b3_rematched = b3_join.loc[lambda x: x.id == x.dict]

b3_unique_rel = pd.concat([b3_unique, b3_rematched])
b3_nodups = b3_unique_rel

In [ ]:
b3_nodups = b3_nodups.drop('dict', axis = 1)
b3_nodups.to_csv('b3_80_wrongids.csv', index = False)

In [ ]:
b3_nodups = pd.read_csv('b3_80_wrongids.csv')

In [ ]:
b3_nodups['b7_umi'] = b3_nodups['umi'].apply(lambda x: True if x in b7_dict else False)

What number of the b3 80% UMIs filtered above are also found in the b7 dictionary?

In [ ]:
print(b3_nodups.loc[b3_nodups['b7_umi'] == True].umi.nunique(), b3_nodups.loc[b3_nodups['b7_umi'] == False].umi.nunique(), b3_nodups.umi.nunique())

In [ ]:
b3_nodups = b3_nodups.loc[b3_nodups['b7_umi'] == True]
b3_nodups['b7_id'] = b3_nodups['umi'].apply(lambda x: b7_dict[x])
b3_nodups['b3_target'] = b3_nodups.id.apply(lambda x: lib[x]['Designed 61-bp target site (37i-24e, AG)'])
b3_nodups['b7_target'] = b3_nodups.b7_id.apply(lambda x: lib[x]['Designed 61-bp target site (37i-24e, AG)'])

In [ ]:
b3_match = b3_nodups.loc[b3_nodups.b3_target == b3_nodups.b7_target]
b3_nomatch = b3_nodups.loc[b3_nodups.b3_target != b3_nodups.b7_target]

In [ ]:
b3match = b3_match.umi.nunique()
b3nomatch = b3_nomatch.umi.nunique()
print(b3match, b3nomatch)
print('around '+str(int(float(b3match)/float(b3match+b3nomatch)*100))+' percent')

In [ ]:
b3_match.to_csv('b3_match.csv', index = False)

In [36]:
b3_match = pd.read_csv(fpath+'/b3_match.csv')

### Dictionary from UMI to ID of unspliced data

In [37]:
b3_dict = pd.Series(b3_match.b7_id.values,index=b3_match.umi).to_dict()

Code below ignores read count 

In [ ]:
b3_unique = pd.merge(b7_uniq_umi, b3_nodups, how = 'inner', on = ['umi'])
b3_unique = b3_unique[b3_unique.apply(lambda x: lib[x.id_x]['Designed 61-bp target site (37i-24e, AG)'] == lib[x.id_y]['Designed 61-bp target site (37i-24e, AG)'], axis=1)]
b3_unique = b3_unique.drop('id_y', axis=1)
b3_unique = b3_unique.drop_duplicates('umi', keep=False)
b3_unique = b3_unique.rename(columns={'id_x':'id'})
b3_unique.id.nunique()

#### Does the b3 analysis remove instances where two umis can match to one id?

NOPE: run code below

In [ ]:
b3_match.loc[b3_match.id == 3563.0]

## __B6 analysis__ [spliced data]

#### Q: Do we need read count information? Yes.

In [38]:
def remove_ggc(umi):
    return umi[:-3]

Below: this chunk of code should maintain read count

In [39]:
b6_nodups = b6.dropna()
b6_nodups.slice_type = b6_nodups.slice_type.apply(lambda x: x.strip())
b6_nodups = b6_nodups[~b6_nodups.umi.str.contains("N")]
b6_nodups.umi = b6_nodups.umi.apply(lambda x: remove_ggc(x))
b6_nodups['b3_umi'] = b6_nodups['umi'].apply(lambda x: True if x in b3_dict else False)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [40]:
b6_umi_checked = b6_nodups.loc[b6_nodups['b3_umi'] == True]

Skip until new code

In [ ]:
b6_umi = pd.Series(b6_umi_checked.id.values,index=b6_umi_checked.umi).to_dict()
# dictionary of mappings from b6 umi to id numbers 

In [ ]:
b6_umi_checked['in_b3'] = b6_umi_checked['umi'].apply(lambda x: b3_dict[x] == b6_umi[x])
# True if the umi maps to the same id as in b3 - else False
b6_umi_checked.loc[b6_umi_checked['in_b3'] == False] 
# not empty right now --> some of these umis do not map to the same id numbers as in b3 --> recombination, need to eliminate them 

In [ ]:
b6_umi_checked['b3_id'] = b6_umi_checked.umi.apply(lambda x: b3_dict[x])

In [ ]:
b6_unique = b6_umi_checked[b6_umi_checked.apply(lambda x: lib[x.id]['Designed 61-bp target site (37i-24e, AG)'] == lib[x.b3_id]['Designed 61-bp target site (37i-24e, AG)'], axis=1)]

In [ ]:
b6_unique = b6_unique.drop('id', axis=1).rename(columns = {'b3_id':'id'})

In [ ]:
b6_unique = pd.read_csv('b6_unique_included_reads.csv')

In [ ]:
b6_unique.to_csv('b6_unique_included_reads.csv', index=False)

New code

In [41]:
b6_unique = b6_umi_checked.loc[b6_umi_checked['b3_umi'] == True]
b6_unique['real_id'] = b6_unique['umi'].apply(lambda x: b3_dict[x])

Trust the b3_dictionary for now: do not isolate b, process all splice types together

In [42]:
b6_unique.shape[0]

184828

In [43]:
b6_unique.groupby('id')['umi'].size().mean()

120.48761408083442

In [ ]:
b6_plot = b6_unique.loc[lambda x: x.id != 4657]

In [ ]:
f = plt.figure()
f.set_size_inches(20,10)
plot = plt.hist(b6_nodups.groupby('umi').size(), bins = 100)

In [45]:
b6_unique.to_csv('b6_match_all.csv', index=False)

Look at all splice types individually

In [ ]:
b6_btype = b6_unique.loc[(b6_unique.slice_type=='b')]
b6_btype['b6_target'] = b6_btype.id.apply(lambda x: lib[x]['Designed 61-bp target site (37i-24e, AG)'])
b6_btype['b3_target'] = b6_btype.real_id.apply(lambda x: lib[x]['Designed 61-bp target site (37i-24e, AG)'])
b6_bmatch = b6_btype.loc[b6_btype.b6_target == b6_btype.b3_target]
b6_bnomatch = b6_btype.loc[b6_btype.b6_target != b6_btype.b3_target]

In [ ]:
b6_bmatch.groupby('real_id').size().mean()

### Q: A normal splice (b type) should have a score of >~70 if it were a nearly normal splice?

In [ ]:
match = b6_bmatch.loc[b6_bmatch.num > 70].shape[0]
nomatch = b6_bnomatch.loc[b6_bnomatch.num > 70].shape[0]
print(match, nomatch)
print(str(int(float(match)/float((match+nomatch))*100))+' percent match with restriction to score above 60')

In [ ]:
plot = plt.hist(b6_unique.num)

In [ ]:
b6_bmatch_dict = pd.Series(b6_bmatch.id.values, index = b6_bmatch.umi).to_dict()

In [ ]:
b6_ctype = b6_unique.loc[b6_unique.slice_type=='c']
b6_ctype['b_present'] = b6_ctype.umi.apply(lambda x: True if x in b6_bmatch_dict else False)
b6_c_and_b = b6_ctype.loc[b6_ctype.b_present == True]

In [ ]:
b6_ctype.loc[b6_ctype.b_present==True].shape[0]

In [ ]:
b6_c_and_b['real_id'] = b6_c_and_b.umi.apply(lambda x: b6_bmatch_dict[x])

In [ ]:
b6_c_and_b.groupby('real_id').size().sort_values(ascending=False)

In [ ]:
b6_cryptic = b6_unique.loc[b6_unique.slice_type=='cryptic']
b6_cryptic['b_present'] = b6_cryptic.umi.apply(lambda x: True if x in b6_bmatch_dict else False)
b6_cryptic_and_b = b6_cryptic.loc[b6_cryptic.b_present == True]

In [ ]:
b6_cryptic.loc[b6_cryptic.b_present ==True].shape[0]

In [ ]:
b6_cryptic_and_b['real_id'] = b6_cryptic_and_b.umi.apply(lambda x: b6_bmatch_dict[x])
b6_cryptic_and_b.groupby('real_id').size().sort_values()

In [ ]:
b6_match = pd.concat([b6_bmatch, b6_c_and_b, b6_cryptic_and_b])
b6_csv = b6_match[['umi','real_id','slice_type']].copy()

In [ ]:
b6_csv

In [ ]:
b6_csv.to_csv('b6_processed.csv',index=False)

## Number of UMIs at each stage of processing

In [ ]:
height = [b7.dropna().shape[0], b3.dropna().shape[0], b6.dropna().shape[0], b7_complete.shape[0], b3_match.shape[0], b6_unique.shape[0], 
          b6_bmatch.shape[0]+b6_c_and_b.shape[0]+b6_cryptic_and_b.shape[0]]
x = [1,2,3,4,5,6,7]
f = plt.figure()
f.set_size_inches(20, 10)
f.suptitle('Raw UMI read count at each stage of cas9 treated data processing (mesc)', y = 1.05)
plt.bar(x, height)
plt.xticks([1,2,3,4,5,6,7], ['b7', 'b3', 'b6','b7 nearly unique matches (>80%)', 'b3 (>80%) and matched with b7 target id','b6 UMIs confirmed by b3 dataset',
                                 'b6 UMIs that match target ids in b3'], rotation = 30)
plt.tight_layout()

In [ ]:
data = [b7.dropna(), b3.dropna(), b6.dropna(), b7_complete, b3_match, b6_unique, pd.concat([b6_bmatch,b6_c_and_b,b6_cryptic_and_b])]
height = np.array(map(lambda x: x.umi.nunique(), data))

In [ ]:
x = [1,2,3,4,5,6,7]
f = plt.figure()
f.set_size_inches(20, 10)
f.suptitle('UMI unique counts at each stage of cas9 treated data processing (mesc)', y = 1.05)
plt.bar(x, height)
plt.xticks([1,2,3,4,5,6,7], ['b7', 'b3', 'b6','b7 nearly unique matches (>80%)', 'b3 (>80%) and matched with b7 target id','b6 UMIs confirmed by b3 dataset',
                                 'b6 UMIs that match target ids in b3'], rotation = 30)
rects = ax.patches
labels = height
for rect, label in zip(rects, labels):
    hght = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2, hght + 5, label,     
    ha='center', va='bottom')
plt.tight_layout()

Below: Code removes read count information

In [ ]:
b6_nodups = b6.drop_duplicates().dropna()
b6_nodups.slice_type = b6_nodups.slice_type.apply(lambda x: x.strip())
b6_nodups = b6_nodups[~b6_nodups.umi.str.contains("N")]
b6_nodups.umi = b6_nodups.umi.apply(lambda x: remove_ggc(x))
b6_unique = pd.merge(b3_unique, b6_nodups, how = 'inner', on = ['umi']) # eliminates UMIs that cannot be used (not matched by b7 and b3)
b6_unique = b6_unique[b6_unique.apply(lambda x: lib[x.id_x]['gene_symbol'] == lib[x.id_y]['gene_symbol'], axis=1)]
b6_unique = b6_unique.drop('id_y', axis=1)

In [ ]:
b6_unique = b6_unique.drop('count', axis=1)

In [ ]:
b6_unique = b6_unique.rename(columns={'id_x':'id'})

Difference in these numbers is due to the fourth column, or the third and fourth column:

In [ ]:
print(b6_unique.umi.nunique(), b6_unique.shape[0])

In [ ]:
b6_umi_dict = pd.Series(b6_unique.id.values,index=b6_unique.umi).to_dict()

In [ ]:
b6_umi = pd.Series(b6_unique.id.values, index=b6_unique.umi).to_dict()

Need to confirm that all the UMIs in the readcount removed set that are duplicated also have the same id - they only vary by the third or fourth column due to cell division duplication 

In [ ]:
b6_unique['uniq_umi'] = b6_unique['umi'].apply(lambda x: b3_dict[x] == b6_umi[x])
b6_unique.loc[b6_unique['uniq_umi'] == False].empty

#### Q: Would two reads from the same UMI, id, splice_type have the same score if they were from the same molecule? 

Most likely yes? If they're from the same molecule, the alignment should be similar, and so should the score. 

### Q: How do we want to store UMI, ID --> splice_type --> fourth column information?

id_dict --> first level key is the id, second key is the splice_type, third is 'num'

In [ ]:
b6_unique = b6_unique.drop('uniq_umi', axis=1) # not needed if b6 is imported from csv --> addition uniq_umi column comes from confirming the duplicated UMIs don't vary by UMI
df2 = b6_unique.groupby(['id','slice_type']).count().reset_index(['slice_type','id']).drop('umi', axis=1)
id_dict = df2.groupby('id')[['slice_type','num']].apply(lambda x: x.set_index(['slice_type']).to_dict(orient='index')).to_dict()

b6_dict --> first key: tuple of umi and id, second key: splice_type, third key: 'num'

In [ ]:
df = b6_unique.groupby(['umi','id','slice_type']).count().reset_index(['slice_type','umi','id'])
b6_dict = df.groupby(['umi','id'])[['slice_type','num']].apply(lambda x: x.set_index(['slice_type']).to_dict(orient='index')).to_dict()

### Q: Assuming all the analysis above is correct so far, which genes have the most number of perfect exon skipping? Print ID and gene symbol

In [ ]:
splice_c_df = df2.loc[df2.slice_type=='c'].sort_values('num', ascending=False)
splice_c_df = splice_c_df.set_index('id',drop=False)
splice_c_df.id.apply(lambda x: gene_lib[x]['gene_symbol'])